In [2]:
import selenium
from bs4 import BeautifulSoup
import requests
import json 

In [3]:
url = "https://www.imdb.com/chart/top/"

In [4]:
r = requests.get(url)

In [5]:
soup = BeautifulSoup(r.content,"html.parser")

In [6]:
tableData = soup.find_all("table", {"class":"chart full-width"})

In [7]:
movieList = (tableData[0].contents)[len(tableData[0].contents)-2]
movieList = movieList.findAll("tr")

In [23]:
with open('data2.json', 'w') as outfile:
    outfile.write("[\n")
    for movie in movieList:

        director = {
            "Fullname" : "",
            "BirthdayMonth" : "" ,
            "BirthdayYear": "",
            "BirthdayPlace" : "",
            "ImageUrl" : "",
            "Biography" : ""
        }
        star = {
            "Name" : "",
            "Image-url" : ""
        }
        data = {
            "Name" : "",
            "Date" : 0,
            "Image-sm" : "",
            "Rating" : "",
            "Description" : "",
            "Director" : director,
            "Categoires" : [],
            "Writers" : [],
            "Stars" : [],
            "Image-lg" : "",
            "Youtube-link" : ""
        }
        
        titleColumn = movie.find_all("td",{"class" : "titleColumn"})
        name = titleColumn[0].a.text
        date = titleColumn[0].span.text.replace("(","").replace(")","")

        data["Name"] = name
        youtube_link = "https://www.youtube.com/results?search_query=" + name + " trailer"
        data["Youtube-link"] = youtube_link
        data["Date"] = date

        poster = movie.find_all("td",{"class" : "posterColumn"})
        image_sm = poster[0].a.img['src']

        data["Image-sm"] = image_sm

        movieDetailLink = "https://www.imdb.com/" + titleColumn[0].a['href']

        requestForMovieDetail = requests.get(movieDetailLink)
        soupMovieDetail = BeautifulSoup(requestForMovieDetail.content,"html.parser")

        rating = movie.find_all("strong")[0].text
        data["Rating"] = rating

        movieDetailLink = "https://www.imdb.com/" + titleColumn[0].a['href']

        requestForMovieDetail = requests.get(movieDetailLink)
        soupMovieDetail = BeautifulSoup(requestForMovieDetail.content,"html.parser")

        try:
            genres = soupMovieDetail.find_all("div" , {"class" : "ipc-chip-list sc-16ede01-4 bMBIRz"})
            for genre in genres[0].find_all("li"):
                data["Categoires"].append(genre.text)
        except:
            continue

        desc = soupMovieDetail.find_all("span",{"class":"sc-16ede01-2 gXUyNh"})[0].text
        data["Description"] = desc


        infoList = soupMovieDetail.find_all("div",{"class":"sc-fa02f843-0 fjLeDR"})
        for info in infoList :
            li_list = info.find_all("li",{"class":"ipc-metadata-list__item"})
            labels = ["Director","Writer","Stars"]
            i = 0
            for liListItem in li_list:
                liItemList = liListItem.find_all("li",{"class":"ipc-inline-list__item"})
                for liItem in liItemList:
                    directorFullname= liItem.a.text
                    # director writes stars
                    if( i == 0 ) :
                        #data["Director"] = a
                        director["Fullname"] = directorFullname
                        try:
                            director_link = "https://www.imdb.com/" + liItem.a['href']
                            requestForDirector = requests.get(director_link)
                            soupDirector = BeautifulSoup(requestForDirector.content,"html.parser")
                            w_image_poster = soupDirector.find_all("img" , {"id" : "name-poster"})
                            director["ImageUrl"] = w_image_poster[0]['src']
                            born_info = soupDirector.find_all("div" , {"id" : "name-born-info"})
                            born = born_info[0].find_all("a")
                            b_count = 0
                            for b in born:
                                if(b_count == 0 ):
                                    director["BirthdayMonth"] = b.text
                                    b_count += 1
                                elif(b_count == 1) :
                                    director["BirthdayYear"] = b.text
                                    b_count += 1
                                elif(b_count == 2) :
                                    director["BirthdayPlace"] = b.text
                                    b_count += 1
                        except:
                            continue
                        
                        see_more = soupDirector.find_all("span" , {"class" : "see-more inline nobr-only"})
                        bio_link = "https://www.imdb.com/" + see_more[0].a['href']
                        requestForDirectorDetail = requests.get(bio_link)
                        soupDirectorDetail = BeautifulSoup(requestForDirectorDetail.content,"html.parser")

                        bio_model = soupDirectorDetail.find_all("div" , {"class" : "soda odd"})
                        bio = bio_model[0].find_all("p")
                        #director["Biography"] = bio[0].text

                    if( i == 1 ) : 
                        writer_name = liItem.a.text
                        data["Writers"].append(writer_name)
                    if( i == 2 ) :
                        try :
                            star_link = "https://www.imdb.com/" + liItem.a['href']
                            star_name = liItem.a.text
                            star["Name"] = star_name

                            requestForStarDetail = requests.get(star_link)
                            soupStarDetail = BeautifulSoup(requestForStarDetail.content,"html.parser")

                            star_image = soupStarDetail.find_all("img" , {"id" : "name-poster"})[0]['src']
                            star["Image-url"] = star_image

                            data["Stars"].append(star)
                        except:
                            continue
                        
                i = i + 1
            data["Director"] = director      
        
        
        try :
            poster_lg = soupMovieDetail.find_all("div",{"class","sc-a658a417-1 bmkYoJ"})
            poster_lg_link = "https://www.imdb.com/" + poster_lg[0].a['href']

            requestForPoster = requests.get(poster_lg_link)
            soupForPoster = BeautifulSoup(requestForPoster.content,"html.parser")

            image_lg = soupForPoster.find_all("div",{"class":"sc-7c0a9e7c-2 bkptFa"})
            image_lg = image_lg[0].img['src']

            data["Image-lg"] = image_lg
        except :
            continue

        json_object = json.dumps(data, indent = 4) 
        outfile.write(json_object)
        outfile.write(",\n")
    outfile.write("\n]")
    outfile.close()